### This notebook shows examples of how to use the cyclops.query API on MIMIC-IV.

## Example 1. Get all patient encounters from 2015 (approximate year of care).

In [ ]:
from sqlalchemy import select

import cyclops.query.util as query_util
from cyclops.query import mimic
from cyclops.query.interface import QueryInterface

patients = mimic.patients(years=2015)
patients.run()
print(f"{len(patients.data)} rows extracted!")

## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [ ]:
patients = mimic.patients(years=[2015])
patients_schizophrenia = mimic.diagnoses(substring="schizophrenia", patients=patients)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")
patients_schizophrenia.data["icd_title"].value_counts()

## Example 3. Advanced - uses regexp_match from sqlalchemy. Get all patient encounters with diagnoses (ICD long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [ ]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(diagnoses.query.c.icd_title.regexp_match("(?=.*schizophrenia)(?=.*chronic)"))
    .subquery()
)
patients_schizophrenia_chronic = diagnoses
patients_schizophrenia_chronic.query = subquery
patients_schizophrenia_chronic.run()
print(f"{len(patients_schizophrenia_chronic.data)} rows extracted!")

## Example 4. Advanced - uses cyclops.query.utils.starts_with helper function. Get all patient encounters with diagnoses (starts with `Paranoid` in ICD long title), in the year 2015.

In [ ]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(query_util.starts_with(diagnoses.query.c.icd_title, "Paranoid"))
    .subquery()
)
patients_schizophrenia = diagnoses
patients_schizophrenia.query = subquery
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [ ]:
patients = mimic.patients()
patients_f209 = mimic.diagnoses(diagnosis_codes=["F209"], patients=patients)
patients_f209.run()
print(f"{len(patients_f209.data)} rows extracted!")

## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [ ]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

## Example 7. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [ ]:
patients = mimic.patients(years=[2015])
patients_vitals = mimic.events(category="Routine Vital Signs", patients=patients)
patients_vitals.run(limit=100)
print(f"{len(patients_vitals.data)} rows extracted!")

## Example 8. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [ ]:
patients = mimic.patients(years=[2009])
patients_hemo_labs = mimic.events(
    category="labs", patients=patients, names="hemoglobin"
)
patients_hemo_labs.run(limit=100)
print(f"{len(patients_hemo_labs.data)} rows extracted!")

## Example 9. Get all lab events for patients from year 2010, that match substring "sodium", limit to 100 rows.

In [ ]:
patients = mimic.patients(years=[2009])
patients_sodium_labs = mimic.events(
    category="labs", patients=patients, substring="sodium"
)
patients_sodium_labs.run(limit=100)
print(f"{len(patients_sodium_labs.data)} rows extracted!")

## Example 10. Get respiratory events for patients from year 2009, limit to 100 rows.

In [ ]:
patients = mimic.patients(years=[2009])
patients_respiratory = mimic.events(category="Respiratory", patients=patients)
patients_respiratory.run(limit=100)
print(f"{len(patients_respiratory.data)} rows extracted!")

## Example 11. Get heart rate measurements of patients with delirium diagnoses, limit to 1000 rows.

In [ ]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium_heart_rate = mimic.events(
    names="Heart Rate", patients=patients_delirium
)
patients_delirium_heart_rate.run(limit=1000)
print(f"{len(patients_delirium_heart_rate.data)} rows extracted!")

## Example 12. Get d_items dimension table, for quick reference.

In [ ]:
event_lookup_table = mimic.get_table("event_labels")
lookup_data = event_lookup_table.run()
print(f"{len(lookup_data)} rows extracted!")